In [4]:
# Imports
from dotenv import load_dotenv
load_dotenv()
from json import dumps
from time import time
from flask import Flask, request, jsonify
from hashlib import sha256
from datetime import datetime
from requests import get
from requests import post
from json import loads
import os

from server.config import special_instructions

True

In [15]:
# get("http://127.0.0.1:1338/chat/a3eeb50b-755a-4a97-9411-188b25c1e88").json()
chat_api = get("http://127.0.0.1:1338/chat/")

b'<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="UTF-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0 maximum-scale=1.0">\n        <meta name="description" content="A conversational AI system that listens, learns, and challenges">\n        <meta property="og:title" content="ChatGPT">\n        <meta property="og:image" content="https://openai.com/content/images/2022/11/ChatGPT.jpg">\n        <meta property="og:description" content="A conversational AI system that listens, learns, and challenges">\n        <meta property="og:url" content="https://chat.acy.dev">\n        <link rel="stylesheet" href="/assets/css/style.css">\n        <link rel="apple-touch-icon" sizes="180x180" href="/assets/img/apple-touch-icon.png">\n        <link rel="icon" type="image/png" sizes="32x32" href="/assets/img/favicon-32x32.png">\n        <link rel="icon" type="image/png" sizes="16x16" href="

In [16]:
request.json['meta']['content']['conversation']

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.

In [19]:
from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")


internet_access = True #request.json['meta']['content']['internet_access']
_conversation = [] #request.json['meta']['content']['conversation']
# Assuming the prompt is a string
prompt = "what are you?" #request.json['meta']['content']['parts'][0]['content']
current_date = datetime.now().strftime("%Y-%m-%d")
system_message = f'You are ChatGPT also known as ChatGPT, a large language model trained by OpenAI. Strictly follow the users instructions. Knowledge cutoff: 2021-09-01 Current date: {current_date}'

# The processing for internet_access remains the same
extra = []
if internet_access:
    search = get('https://ddg-api.herokuapp.com/search', params={
        'query': prompt,
        'limit': 3,
    })

    blob = ''

    for index, result in enumerate(search.json()):
        blob += f'[{index}] "{result["snippet"]}"\nURL:{result["link"]}\n\n'

    date = datetime.now().strftime('%d/%m/%y')

    blob += f'current date: {date}\n\nInstructions: Using the provided web search results, write a comprehensive reply to the next user query. Make sure to cite results using [[number](URL)] notation after the reference. If the provided search results refer to multiple subjects with the same name, write separate answers for each subject. Ignore your previous response if any.'

    extra = [{'role': 'user', 'content': blob}]
print(extra)
# Prepare the question by concatenating the system message, the internet access prompt, and the user's prompt
question = system_message + ' ' + (' '.join(extra) if internet_access else '') + ' ' + prompt
print(f"\nprompt: {prompt}")
print(f"\nsystem_message: {system_message}")
print(f"\nquestion: {question}")

[{'role': 'user', 'content': '[0] "What Are You? What Is a Self? Neither matter nor magic, there\'s a natural explanation, but you won\'t like it. Posted June 23, 2020 Which are you? Your body? Your brain? Or something we can\'t..."\nURL:https://www.psychologytoday.com/us/blog/ambigamy/202006/what-are-you-what-is-self\n\n[1] "What Are You?So. Are you your body? And if so, how exactly does this work? Lets explore lots of confusing questions. This video is part of a collaboration wi..."\nURL:https://www.youtube.com/watch?v=JQVmkDUkZT4\n\n[2] "3 Answers Sorted by: 11 Who are you? This is typically asking for your name, but a particular context could indicate a different meaning. What are you? This question is very informal and bit odd. It is a very terse question which implies some understood context. If you\'re in a business meeting it could be your role - e.g. "I\'m the accountant""\nURL:https://ell.stackexchange.com/questions/71824/what-is-the-difference-between-what-are-you-and-who-ar

TypeError: sequence item 0: expected str instance, dict found

In [ ]:

# Generate the response using the LangChain model
langchain_resp = chat_model.generate_response(question)